## **YOLO v5 화장실 표지판 이미지 학습**
1. 데이터 수집 : image data, label data (image의 정보를 저장한 txt 파일), yaml 파일 (이미지 경로, class 개수, class 이름의 정보가 담김)
2. 데이터를 이용해 학습시킨 뒤 weight 파일(.pt)을 얻음
3. weight 파일로 detect를 함


## 실제로 돌려보고싶다면?
1. 먼저 1번과 2번 셀 돌린다. 돌리면 yolov5라는 새로운 파일이 생긴다.
2. 내가 직접 새로운 파일을 만든다. 파일 이름은 dataset
3. 이름이 dataset인 파일에 yaml, images, labels 3가지를 모두 넣는다.
4. 나머지 다 돌려버린다.
5. best.pt가 생긴다. 확인은 yolob5 -> runs -> train -> toilet_yolov5s_results -> weights -> best.pt 여기에 있다. 이거 다운받는다.


## 확인을 해보고 싶다면 
best.pt를 c드라이브에 있는 파일인 yolov5에 넣음

그다음에 아나콘다 환경에서

1. activate 가상환경이름
2. cd yolov5
3. python detect.py --weights best.pt --source 0

ex)

(base) C:\Users\user>activate yolov5
여기서 yolov5는 가상환경 이름임

(yolov5) C:\Users\user>cd yolov5

(yolov5) C:\Users\user\yolov5>python detect.py --weights toilet.pt --source 0
여기서 toilet.pt는 best.pt를 이름변경한 것

In [3]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [4]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


yaml 파일을 수정하기 위해 정보를 열람

In [5]:
%cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['toilet']

In [6]:
%cd /
from glob import glob

img_list = glob('/content/dataset/*.jpg') #다운 받은 이미지

print(len(img_list))

/
176


다운받은 이미지를 training image(90%)와 validate image(10%)로 나눔

In [7]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.1, random_state=2000)

print(len(train_img_list), len(val_img_list))

158 18


training image, validate image의 경로를 txt 파일로 저장

In [8]:
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

yaml 파일 수정

In [9]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)  #수정 이전

data['train'] = '/content/dataset/train.txt'  #경로를 수정
data['val'] = '/content/dataset/val.txt'

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)  #수정 이후

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['toilet']}
{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'nc': 1, 'names': ['toilet']}


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


train을 통해 weight를 얻음

In [11]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name toilet_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=toilet_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-388-gf1214f2 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou